In [ ]:
%run local_prot_vec.py

In [ ]:
%run main.py --yaml_path yamls/protein_vec.yaml

In [ ]:
import torch

def are_models_equal(model1, model2):
    state_dict1 = model1.state_dict()
    state_dict2 = model2.state_dict()
    
    if len(state_dict1) != len(state_dict2):
        print('Length')
        return False
    
    for key in state_dict1:
        if key not in state_dict2:
            print('Key')
            return False
        
        if not torch.equal(state_dict1[key], state_dict2[key]):
            print('Equal')
            return False
    
    return True


if are_models_equal(disk_model.cpu(), hf_model.cpu()):
    print("The models are equal.")
else:
    print("The models are not equal.")

In [ ]:
from data.load_data import get_datasets_test_triplet
from transformers import T5Tokenizer

In [ ]:
args = {
    'data_paths': ['lhallee/triplets'],
    'domains': ['[EC]', '[CC]', '[MF]', '[BP]', '[CC]', '[CC]', '[IP]'],
    'new_special_tokens': False,
    'max_length':512, 
    'p_col': 'positives',
    'a_col': 'anchors',
    'n_col': 'negatives',
    'label_col': 'aspects',
    'model_type': 'ProteinVec'
}

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('lhallee/ProteinVec')

In [ ]:
triplet_datasets = get_datasets_test_triplet(args, tokenizer)

In [ ]:
ec = triplet_datasets[0]
ec

In [ ]:
ec[1][4]

In [ ]:
import torch
import torch.nn as nn
from models.modeling_moesm import EsmExpert


class SentenceEnforcedSwitchMoeBlock(nn.Module): ### Test
    def __init__(self, config, expert):
        """
        Sentence level MoE, single expert chosen
        """
        super().__init__()
        self.hidden_dim = config.hidden_size
        self.num_experts = config.num_experts
        self.experts = nn.ModuleList([expert(config) for _ in range(self.num_experts)])

    def forward(self, hidden_states: torch.Tensor, router_labels: torch.tensor) -> torch.Tensor:
        # (batch, seq_len, hidden_size), (batch,) -> from 0 to num_experts-1
        sorted_indices = torch.argsort(router_labels) # sort in order of expert idx
        hidden_states = hidden_states[sorted_indices] # apply sort
        router_labels = router_labels[sorted_indices] # apply sort
        expert_idxs = torch.unique(router_labels) # find all experts needed
        grouped_hidden_states = torch.split(hidden_states, tuple(torch.bincount(router_labels))) # split sorted hidden_states

        expert_outputs = []
        for idx, group in zip(expert_idxs, grouped_hidden_states):
            expert_output = self.experts[idx](group) # sne batched groups to their experts
            expert_outputs.append(expert_output)

        concatenated_outputs = torch.cat(expert_outputs, dim=0)
        final_hidden_states = concatenated_outputs[torch.argsort(sorted_indices)] # put back to original order
        return final_hidden_states  # (batch, sequence_length, hidden_dim)
    
class Config:
    def __init__(self):
        self.hidden_size = 128
        self.num_experts = 4
        self.intermediate_size = 256
        self.hidden_dropout_prob = 0.0


config = Config()
moe_block = SentenceEnforcedSwitchMoeBlock(config, EsmExpert)




In [ ]:
# Generate random tensors for testing
batch_size = 8
seq_length = 16
hidden_size = config.hidden_size

hidden_states = torch.randn(batch_size, seq_length, hidden_size)
router_labels = torch.randint(0, config.num_experts, (batch_size,))

# Test the forward pass of the SentenceEnforcedSwitchMoeBlock
output = moe_block(hidden_states, router_labels)

# Print the shapes of input and output tensors
print("Input hidden states shape:", hidden_states.shape)
print("Router labels shape:", router_labels.shape)
print("Output shape:", output.shape)

In [ ]:
router_labels = torch.tensor([2, 0, 1, 2, 1])
print(router_labels)
sorted_indices = torch.argsort(router_labels)

sorted_labels = router_labels[sorted_indices]

unsorted_indices = torch.argsort(sorted_indices)

router_labels = sorted_labels[unsorted_indices]
print(router_labels)

In [1]:
from utils import get_yaml
from models.load_model import load_models

In [2]:
class arguments:
    yaml_path = 'yamls/MOE/moesm_double.yaml'

args = arguments()
yargs = get_yaml(args.yaml_path)
for key, value in yargs['general_args'].items(): # copy yaml config into args
    setattr(args, key, value)
for key, value in yargs['training_args'].items():
    setattr(args, key, value)
model, tokenizer = load_models(args) # if eval and skip, not needed

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights differ
All weights match
Model loaded in  0.05 minutes
518.2 million total parameters
473.9 million effective parameters
Approximately 1.93 GB of memory in fp32

MoEsmVec(
  (base): T5EncoderModel(
    (shared): Embedding(144, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(144, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(64, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (1): T5LayerFF(
          

In [3]:
import torch

def generate_random_inputs(vocab_size, max_length, batch_size=1):
    base_a_ids = torch.randint(0, vocab_size, (batch_size, max_length))
    base_b_ids = torch.randint(0, vocab_size, (batch_size, max_length))
    base_a_mask = torch.randint(0, 2, (batch_size, max_length))
    base_b_mask = torch.randint(0, 2, (batch_size, max_length))
    plm_a_ids = torch.randint(0, vocab_size, (batch_size, max_length))
    plm_b_ids = torch.randint(0, vocab_size, (batch_size, max_length))
    plm_a_mask = torch.randint(0, 2, (batch_size, max_length))
    plm_b_mask = torch.randint(0, 2, (batch_size, max_length))
    return base_a_ids, base_b_ids, base_a_mask, base_b_mask, plm_a_ids, plm_b_ids, plm_a_mask, plm_b_mask

def print_tensor_flow(model, input_tensors):
    names, weights, inputs, outputs = [], [], [], []

    def hook_fn(module, input, output):
        if hasattr(module, 'weight'):
            names.append(module.__class__.__name__)
            weights.append(tuple(module.weight.shape))
            inputs.append(tuple(input[0].shape))
            outputs.append(tuple(output.shape))
        elif 'pool' in module.__class__.__name__.lower():
            names.append(module.__class__.__name__)
            weights.append('None')
            inputs.append(tuple(input[0].shape))
            outputs.append(tuple(output.shape))

    handles = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Module):
            handles.append(module.register_forward_hook(hook_fn))

    with torch.no_grad():
        out = model(*input_tensors)

    for handle in handles:
        handle.remove()

    print('Tensor flow')
    print('{:<25} {:<25} {:<25} {:<25}'.format('Layer', 'Weight Size', 'Input size', 'Output size'))
    print('-' * 100)
    for name, weight, input, output in zip(names, weights, inputs, outputs):
        print('{:<25} {:<25} {:<25} {:<25}'.format(str(name), str(weight), str(input), str(output)))

# Example usage
vocab_size = 20
max_length = 8
batch_size = 1

input_tensors = generate_random_inputs(vocab_size, max_length, batch_size)
print_tensor_flow(model, input_tensors)

Tensor flow
Layer                     Weight Size               Input size                Output size              
----------------------------------------------------------------------------------------------------
Embedding                 (144, 768)                (1, 8)                    (1, 8, 768)              
T5LayerNorm               (768,)                    (1, 8, 768)               (1, 8, 768)              
Linear                    (768, 768)                (1, 8, 768)               (1, 8, 768)              
Linear                    (768, 768)                (1, 8, 768)               (1, 8, 768)              
Linear                    (768, 768)                (1, 8, 768)               (1, 8, 768)              
Embedding                 (64, 12)                  (8, 8)                    (8, 8, 12)               
Linear                    (768, 768)                (1, 8, 768)               (1, 8, 768)              
T5LayerNorm               (768,)                    (1,

In [2]:
from transformers import AutoTokenizer

In [4]:
esm = AutoTokenizer.from_pretrained('facebook/esm2_t6_8M_UR50D')
ankh = AutoTokenizer.from_pretrained('lhallee/ankh_base_encoder')

tokenizer_config.json:   0%|          | 0.00/23.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/31.2k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

In [5]:
seq = 'MQIFVKTLTGKTITLEVEPSETIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG'
print(esm(seq).input_ids)
print(ankh(seq).input_ids)

[0, 20, 16, 12, 18, 7, 15, 11, 4, 11, 6, 15, 11, 12, 11, 4, 9, 7, 9, 14, 8, 9, 11, 12, 9, 17, 7, 15, 5, 15, 12, 16, 13, 15, 9, 6, 12, 14, 14, 13, 16, 16, 10, 4, 12, 18, 5, 6, 15, 16, 4, 9, 13, 6, 10, 11, 4, 8, 13, 19, 17, 12, 16, 15, 9, 8, 11, 4, 21, 4, 7, 4, 10, 4, 10, 6, 6, 2]
[19, 16, 12, 15, 6, 14, 11, 4, 11, 5, 14, 11, 12, 11, 4, 9, 6, 9, 13, 7, 9, 11, 12, 9, 17, 6, 14, 3, 14, 12, 16, 10, 14, 9, 5, 12, 13, 13, 10, 16, 16, 8, 4, 12, 15, 3, 5, 14, 16, 4, 9, 10, 5, 8, 11, 4, 7, 10, 18, 17, 12, 16, 14, 9, 7, 11, 4, 20, 4, 6, 4, 8, 4, 8, 5, 5, 1]
